In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from zero_helpers.imports import * 
from zero_liftsim.simmanager import SimulationManager
from zero_liftsim.helpers import base_config
from zero_liftsim.experience import AgentRideLoopExperience
from zero_liftsim.helpers import load_asset_template
from zero_liftsim import helpers as hp
from zero_liftsim.lift import Lift

import inspect

# Run Simulation

In [15]:
cfg = base_config()
cfg["SimulationManager"]["__init__"].update({"n_agents": 3, "lift_capacity": 2})
manager = SimulationManager(cfg)
result = manager.run(runtime_minutes=60)
print(result)

{'total_rides': 11, 'average_wait': 3.090909090909091, 'agents': [Agent(1) eminent-following de26358e-206f-45c8-81c9-c693e011f379, Agent(2) seemly-habit 57d63cb6-9f71-4d9f-b54e-6fa480860b03, Agent(3) caring-official c9d4351e-6de5-4bf0-8234-11f5254b78a6]}


In [16]:
lift = manager.lift
lift.total_chairs()

50

## Retrieve Data from Simulation

In [17]:
# get rideloop explogs and agent event log
exp_log_data = manager.retrieve_data()
exp = exp_log_data['exp_rideloop']
log = exp_log_data['agent_log']

# Dev

In [18]:
# sample agent and subset exp/log to e/l for agent subset of those
m = log['event'] == 'ride_complete'
event_ride_comp = log[m].sample().iloc[0].to_dict()
agent = manager.lookup_agent(event_ride_comp['agent_uuid'])
l = log[log['agent_uuid'] == agent.agent_uuid]
idx = (exp['time'] - event_ride_comp['time']).dt.total_seconds().abs().idxmin()
e = exp.loc[idx].to_dict()

In [19]:
e

{'time': Timestamp('2025-03-12 09:19:00'),
 'exp_id': '972a4e14-5b20-4ffa-beec-e56654dac66b',
 'return_event_uuid': '2296145a-3ff4-4d8e-804b-7eb954f78dde',
 'agent_uuid': 'de26358e-206f-45c8-81c9-c693e011f379',
 'agent_uuid_codename': 'eminent-following',
 'time_spent_ride_lift': 6.752761098100441,
 'time_spent_traverse_down_mountain': 5.4936342815300785,
 'time_spent_in_queue': 0,
 'time_spent_total': 12.24639537963052}

In [26]:
x = agent.traceback_experience(e['exp_id'])
print(x)

# Experience Log 

exp_id: 972a4e14-5b20-4ffa-beec-e56654dac66b
agent_uuid: de26358e-206f-45c8-81c9-c693e011f379
return_event_uuid: 2296145a-3ff4-4d8e-804b-7eb954f78dde

*** 


time_spent_in_queue: 0
time_spent_ride_lift: 6.752761098100441
time_spent_traverse_down_mountain: 5.4936342815300785



In [29]:
l.columns

Index(['return_event_uuid', 'time', 'time_offset', 'event', 'agent_id',
       'agent_uuid', 'agent_uuid_codename', 'queue_length', 'lift_state',
       'wait_time', 'wait_time_readable'],
      dtype='object')